# 한국어: 인문학, 사회과학

선정된 과잉 단어들(excess_words_for_labeling_final.xlxs)이 실제 초록에서 연도별로 얼마나 사용되는지 추이 파악하는 코드

- 측정 지표
1. excess_count 총 출현 횟수(중복 포함한 전체 사용량)
2. excess_unique 고유 단어 개수 (몇 종류의 과잉 단어들을 쓰는가?)
- found_words에 같은 과잉 단어가 여러 번 나오더라도 종류로는 1개로 침. (예시: "시사한다"가 10번 나와도 unique_count에서는 1번만 카운트 되는 것임.)
3. excess_ratio 과잉 단어,어절 수
- 예를 들어, 초록이 100어절에 과잉 단어가 4번 나온 거면 ratio는 4퍼센트인 것.

과잉 단어 N개 이상 사용한 초록이 몇 퍼센트인가? (현재 1, 3, 5, 10, 15, 20개를 기준으로 함.)
연도별 평균, 중앙값, 분포 변화를 보고 있음.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (Windows의 경우)
plt.rcParams['axes.unicode_minus'] = False

# 1. 과잉 단어 파일 경로
# 필요한 파일: (H 혹은 SS)_excess_words_top_combined_for_labeling_final.xlsx
EXCESS_WORDS_FILE = "/content/SS_excess_words_top_combined_for_labeling_final.xlsx"

# 2. 원본 초록 데이터 파일 경로 (전처리된 CSV/Excel)
# 필요한 파일: (H 혹은 SS)_kor_abstract_clean.csv
ABSTRACTS_FILE = "/content/SS_kor_abstract_clean.csv"

# 3. 출력된 폴더
OUTPUT_DIR = Path("/content/excess_analysis")

# 4. 분석 기준 (과잉 단어 N개 이상)
THRESHOLDS = [1, 3, 5, 10, 15, 20]

# 5. 분야명 (그래프 제목용)
FIELD_NAME = "Humanities"  # 또는 "Social Sciences"


def load_excess_words(filepath):
    """과잉 단어 목록 로드"""
    df = pd.read_excel(filepath)
    words = set(df['word'].tolist())
    print(f"과잉 단어 {len(words)}개 로드 완료")
    return words, df


def load_abstracts(filepath, encoding='utf-8-sig'):
    """초록 데이터 로드"""
    if filepath.endswith('.xlsx') or filepath.endswith('.xls'):
        df = pd.read_excel(filepath)
    else:
        df = pd.read_csv(filepath, encoding=encoding)
    print(f"초록 데이터 {len(df)}건 로드 완료")
    return df


def count_excess_in_text(text, excess_words):
    if pd.isna(text) or not isinstance(text, str):
        return 0, 0, []

    found_words = [] # 발견된 단어들
    for word in excess_words:
        cnt = text.count(word) # 총 출현 횟수(같은 단어가 여러 번 나오면 다 셈.)
        if cnt > 0:
            found_words.extend([word] * cnt)

    count = len(found_words)
    unique_count = len(set(found_words)) # 고유 과잉 단어 개수

    return count, unique_count, list(set(found_words))


# 모든 초록 돌면서 각각에 대하여 과잉 단어 개수, 텍스트 길이(어절 수), 과잉 비율=과잉 단어 수/전체 어절 수
def analyze_all_abstracts(df, excess_words, year_col='발행년도', abstract_col='초록'):


    results = []
    total = len(df)

    for idx, row in df.iterrows():
        if idx % 10000 == 0:
            print(f"  진행: {idx:,} / {total:,} ({idx/total*100:.1f}%)")

        year = row[year_col]
        text = row[abstract_col]

        count, unique_count, found = count_excess_in_text(text, excess_words)

        # 텍스트 길이 (대략적인 어절 수)
        text_len = len(str(text).split()) if pd.notna(text) else 0
        ratio = count / text_len if text_len > 0 else 0

        results.append({
            'year': year,
            'excess_count': count,
            'excess_unique': unique_count,
            'excess_ratio': ratio,
            'text_length': text_len
        })

    print(f"  완료: {total:,}건 분석")
    return pd.DataFrame(results)


def calculate_threshold_stats(analysis_df, thresholds):
    """방법 2: N개 이상 과잉 단어 사용한 초록 비율 계산"""
    years = sorted(analysis_df['year'].unique())

    stats = []
    for year in years:
        year_data = analysis_df[analysis_df['year'] == year]
        total = len(year_data)

        row = {'year': year, 'total_abstracts': total}

        for n in thresholds:
            count = (year_data['excess_unique'] >= n).sum()
            ratio = count / total * 100 if total > 0 else 0
            row[f'n>={n}_count'] = count
            row[f'n>={n}_ratio'] = ratio

        stats.append(row)

    return pd.DataFrame(stats)


def plot_threshold_trends(stats_df, thresholds, output_path, field_name=""):
    """방법 2 시각화: N개 이상 사용 비율 추이"""
    fig, ax = plt.subplots(figsize=(12, 6))

    years = stats_df['year'].values
    colors = plt.cm.viridis(np.linspace(0, 0.8, len(thresholds)))

    for i, n in enumerate(thresholds):
        col = f'n>={n}_ratio'
        ax.plot(years, stats_df[col], marker='o', label=f'{n}+ words',
                color=colors[i], linewidth=2, markersize=5)

    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Percentage of Abstracts (%)', fontsize=12)
    ax.set_title(f'[{field_name}] Abstracts Using N+ Excess Words Over Time', fontsize=14)
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_xticks(years[::2])

    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"저장: {output_path}")


def plot_score_distribution(analysis_df, output_path, field_name=""):
    """방법 3 시각화: 연도별 과잉 점수 분포 (박스플롯)"""
    years = sorted(analysis_df['year'].unique())
    recent_years = years[-10:] if len(years) > 10 else years
    recent_data = analysis_df[analysis_df['year'].isin(recent_years)]

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # (1) 고유 과잉 단어 개수 분포
    ax1 = axes[0]
    data_unique = [recent_data[recent_data['year'] == y]['excess_unique'].values
                   for y in recent_years]
    bp1 = ax1.boxplot(data_unique, labels=recent_years, patch_artist=True)
    for patch in bp1['boxes']:
        patch.set_facecolor('lightblue')
    ax1.set_xlabel('Year', fontsize=11)
    ax1.set_ylabel('Unique Excess Words per Abstract', fontsize=11)
    ax1.set_title(f'[{field_name}] Distribution of Excess Word Count', fontsize=12)
    ax1.tick_params(axis='x', rotation=45)

    # (2) 과잉 단어 비율 분포
    ax2 = axes[1]
    data_ratio = [recent_data[recent_data['year'] == y]['excess_ratio'].values * 100
                  for y in recent_years]
    bp2 = ax2.boxplot(data_ratio, labels=recent_years, patch_artist=True)
    for patch in bp2['boxes']:
        patch.set_facecolor('lightcoral')
    ax2.set_xlabel('Year', fontsize=11)
    ax2.set_ylabel('Excess Word Ratio (%)', fontsize=11)
    ax2.set_title(f'[{field_name}] Distribution of Excess Word Ratio', fontsize=12)
    ax2.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"저장: {output_path}")


def plot_yearly_mean_trend(analysis_df, output_path, field_name=""):
    """연도별 평균 과잉 단어 사용량 추이"""
    yearly_stats = analysis_df.groupby('year').agg({
        'excess_count': ['mean', 'std', 'median'],
        'excess_unique': ['mean', 'std', 'median'],
        'excess_ratio': ['mean', 'std', 'median']
    }).reset_index()

    yearly_stats.columns = ['year',
                            'count_mean', 'count_std', 'count_median',
                            'unique_mean', 'unique_std', 'unique_median',
                            'ratio_mean', 'ratio_std', 'ratio_median']

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    years = yearly_stats['year'].values

    # (1) 평균 고유 과잉 단어 개수
    ax1 = axes[0]
    ax1.plot(years, yearly_stats['unique_mean'], 'b-o', linewidth=2,
             markersize=6, label='Mean')
    ax1.plot(years, yearly_stats['unique_median'], 'b--s', linewidth=1.5,
             markersize=4, alpha=0.7, label='Median')

    ax1.set_xlabel('Year', fontsize=11)
    ax1.set_ylabel('Unique Excess Words', fontsize=11)
    ax1.set_title(f'[{field_name}] Mean Excess Word Count per Abstract', fontsize=12)
    ax1.legend(fontsize=9)
    ax1.grid(True, alpha=0.3)

    n_samples = analysis_df.groupby('year').size().values
    # 음영 부분
    ci_unique = 1.96 * yearly_stats['unique_std'] / np.sqrt(n_samples)
    ax1.fill_between(years,
                 yearly_stats['unique_mean'] - ci_unique,
                 yearly_stats['unique_mean'] + ci_unique,
                 alpha=0.2)
    ax1.set_xticks(years[::2])

    # (2) 평균 과잉 단어 비율
    ax2 = axes[1]
    ax2.plot(years, yearly_stats['ratio_mean'] * 100, 'r-o', linewidth=2,
             markersize=6, label='Mean')
    ax2.plot(years, yearly_stats['ratio_median'] * 100, 'r--s', linewidth=1.5,
             markersize=4, alpha=0.7, label='Median')

    ax2.set_xlabel('Year', fontsize=11)
    ax2.set_ylabel('Excess Word Ratio (%)', fontsize=11)
    ax2.set_title(f'[{field_name}] Mean Excess Word Ratio per Abstract', fontsize=12)
    ax2.legend(fontsize=9)
    ax2.grid(True, alpha=0.3)

    ci_ratio = 1.96 * yearly_stats['ratio_std'] / np.sqrt(n_samples)
    # 음영 부분
    ax2.fill_between(years,
                     (yearly_stats['ratio_mean'] - yearly_stats['ratio_std']) * 100,
                     (yearly_stats['ratio_mean'] + yearly_stats['ratio_std']) * 100,
                     alpha=0.2, color='red')
     ax2.set_xticks(years[::2])

    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"저장: {output_path}")

    return yearly_stats


def main():
    print("=" * 60)
    print("과잉 단어 사용 추이 분석")
    print("=" * 60)

    # 출력 폴더 생성
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    # 1. 과잉 단어 로드
    print("\n[1] 과잉 단어 로드 중...")
    excess_words, excess_df = load_excess_words(EXCESS_WORDS_FILE)

    # 2. 초록 데이터 로드
    print("\n[2] 초록 데이터 로드 중...")
    abstracts_df = load_abstracts(ABSTRACTS_FILE)

    # 3. 분석 실행
    print("\n[3] 초록별 과잉 단어 분석...")
    analysis_df = analyze_all_abstracts(abstracts_df, excess_words)

    # 분석 결과 저장
    analysis_df.to_csv(OUTPUT_DIR / 'abstract_excess_analysis.csv',
                       index=False, encoding='utf-8-sig')
    print(f"  → 저장: {OUTPUT_DIR / 'abstract_excess_analysis.csv'}")

    # 4. 방법 2: N개 이상 사용 비율
    print("\n[4] 임계값별 비율 계산...")
    stats_df = calculate_threshold_stats(analysis_df, THRESHOLDS)
    stats_df.to_csv(OUTPUT_DIR / 'threshold_stats.csv',
                    index=False, encoding='utf-8-sig')
    print(stats_df.to_string())

    # 5. 시각화
    print("\n[5] 시각화...")
    plot_threshold_trends(stats_df, THRESHOLDS,
                          OUTPUT_DIR / 'threshold_trends.png', FIELD_NAME)
    plot_score_distribution(analysis_df,
                            OUTPUT_DIR / 'score_distribution.png', FIELD_NAME)
    yearly_stats = plot_yearly_mean_trend(analysis_df,
                                          OUTPUT_DIR / 'yearly_mean_trend.png', FIELD_NAME)

    # 연도별 통계 저장
    yearly_stats.to_csv(OUTPUT_DIR / 'yearly_stats.csv',
                        index=False, encoding='utf-8-sig')

    print("\n" + "=" * 60)
    print("✅ 분석 완료!")
    print(f"   결과 저장 위치: {OUTPUT_DIR}")
    print("=" * 60)

    # 요약 통계 출력
    print("\n[요약]")
    print(f"- 전체 초록 수: {len(analysis_df):,}")
    print(f"- 과잉 단어 1개 이상 사용한 초록: {(analysis_df['excess_unique'] >= 1).sum():,} "
          f"({(analysis_df['excess_unique'] >= 1).mean()*100:.1f}%)")
    print(f"- 과잉 단어 5개 이상 사용한 초록: {(analysis_df['excess_unique'] >= 5).sum():,} "
          f"({(analysis_df['excess_unique'] >= 5).mean()*100:.1f}%)")
    print(f"- 평균 과잉 단어 개수: {analysis_df['excess_unique'].mean():.2f}")

    # 2024년 vs 2020년 비교
    if 2024 in analysis_df['year'].values and 2020 in analysis_df['year'].values:
        y2020 = analysis_df[analysis_df['year'] == 2020]['excess_unique'].mean()
        y2024 = analysis_df[analysis_df['year'] == 2024]['excess_unique'].mean()
        print(f"\n- 2020년 평균: {y2020:.2f} → 2024년 평균: {y2024:.2f} "
              f"(변화: {(y2024-y2020)/y2020*100:+.1f}%)")


if __name__ == '__main__':
    main()

# 영어: 인문학, 사회과학

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (Windows의 경우)
# plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


# 1. 과잉 단어 파일 경로
# 필요한 파일: (H 혹은 SS)_excess_words_top_labeled.xlsx
EXCESS_WORDS_FILE = "/content/SS_excess_words_top_labeled.xlsx"

# 2. 원본 초록 데이터 파일 경로 (전처리된 CSV/Excel)
# 필요한 파일: (H 혹은 SS)_eng_abstract_clean.csv
ABSTRACTS_FILE = "/content/SS_eng_abstract_clean.csv"

# 3. 출력된 폴더
OUTPUT_DIR = Path("/content/excess_analysis")

# 4. 분석 기준 (과잉 단어 N개 이상)
THRESHOLDS = [1, 3, 5, 10, 15, 20]

# 5. 분야명 (그래프 제목용)
FIELD_NAME = "Humanities"  # 또는 "Social Sciences"


# ============================================================
# 함수 정의
# ============================================================

def load_excess_words(filepath):
    """과잉 단어 목록 로드"""
    df = pd.read_excel(filepath)
    words = set(df['word'].tolist())
    print(f"과잉 단어 {len(words)}개 로드 완료")
    return words, df


def load_abstracts(filepath, encoding='utf-8-sig'):
    """초록 데이터 로드"""
    if filepath.endswith('.xlsx') or filepath.endswith('.xls'):
        df = pd.read_excel(filepath)
    else:
        df = pd.read_csv(filepath, encoding=encoding)
    print(f"초록 데이터 {len(df)}건 로드 완료")
    return df


def count_excess_in_text(text, excess_words):
    """
    텍스트에서 과잉 단어 출현 횟수 계산 (단순 문자열 매칭)

    Returns:
        - count: 과잉 단어 총 출현 횟수 (중복 포함)
        - unique_count: 사용된 고유 과잉 단어 개수
        - found_words: 발견된 과잉 단어 리스트
    """
    if pd.isna(text) or not isinstance(text, str):
        return 0, 0, []

    found_words = []
    for word in excess_words:
        cnt = text.count(word)
        if cnt > 0:
            found_words.extend([word] * cnt)

    count = len(found_words)
    unique_count = len(set(found_words))

    return count, unique_count, list(set(found_words))


def analyze_all_abstracts(df, excess_words, year_col='발행년도', abstract_col='영어초록'):
    """전체 초록 데이터 분석"""
    print("\n초록 분석 중...")

    results = []
    total = len(df)

    for idx, row in df.iterrows():
        if idx % 10000 == 0:
            print(f"  진행: {idx:,} / {total:,} ({idx/total*100:.1f}%)")

        year = row[year_col]
        text = row[abstract_col]

        count, unique_count, found = count_excess_in_text(text, excess_words)

        # 텍스트 길이 (대략적인 어절 수)
        text_len = len(str(text).split()) if pd.notna(text) else 0
        ratio = count / text_len if text_len > 0 else 0

        results.append({
            'year': year,
            'excess_count': count,
            'excess_unique': unique_count,
            'excess_ratio': ratio,
            'text_length': text_len
        })

    print(f"  완료: {total:,}건 분석")
    return pd.DataFrame(results)


def calculate_threshold_stats(analysis_df, thresholds):
    """방법 2: N개 이상 과잉 단어 사용한 초록 비율 계산"""
    years = sorted(analysis_df['year'].unique())

    stats = []
    for year in years:
        year_data = analysis_df[analysis_df['year'] == year]
        total = len(year_data)

        row = {'year': year, 'total_abstracts': total}

        for n in thresholds:
            count = (year_data['excess_unique'] >= n).sum()
            ratio = count / total * 100 if total > 0 else 0
            row[f'n>={n}_count'] = count
            row[f'n>={n}_ratio'] = ratio

        stats.append(row)

    return pd.DataFrame(stats)


def plot_threshold_trends(stats_df, thresholds, output_path, field_name=""):
    """방법 2 시각화: N개 이상 사용 비율 추이"""
    fig, ax = plt.subplots(figsize=(12, 6))

    years = stats_df['year'].values
    colors = plt.cm.viridis(np.linspace(0, 0.8, len(thresholds)))

    for i, n in enumerate(thresholds):
        col = f'n>={n}_ratio'
        ax.plot(years, stats_df[col], marker='o', label=f'{n}+ words',
                color=colors[i], linewidth=2, markersize=5)

    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Percentage of Abstracts (%)', fontsize=12)
    ax.set_title(f'[{field_name}] Abstracts Using N+ Excess Words Over Time', fontsize=14)
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_xticks(years[::2])

    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"저장: {output_path}")


def plot_score_distribution(analysis_df, output_path, field_name=""):
    """방법 3 시각화: 연도별 과잉 점수 분포 (박스플롯)"""
    years = sorted(analysis_df['year'].unique())
    recent_years = years[-10:] if len(years) > 10 else years
    recent_data = analysis_df[analysis_df['year'].isin(recent_years)]

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # (1) 고유 과잉 단어 개수 분포
    ax1 = axes[0]
    data_unique = [recent_data[recent_data['year'] == y]['excess_unique'].values
                   for y in recent_years]
    bp1 = ax1.boxplot(data_unique, labels=recent_years, patch_artist=True)
    for patch in bp1['boxes']:
        patch.set_facecolor('lightblue')
    ax1.set_xlabel('Year', fontsize=11)
    ax1.set_ylabel('Unique Excess Words per Abstract', fontsize=11)
    ax1.set_title(f'[{field_name}] Distribution of Excess Word Count', fontsize=12)
    ax1.tick_params(axis='x', rotation=45)

    # (2) 과잉 단어 비율 분포
    ax2 = axes[1]
    data_ratio = [recent_data[recent_data['year'] == y]['excess_ratio'].values * 100
                  for y in recent_years]
    bp2 = ax2.boxplot(data_ratio, labels=recent_years, patch_artist=True)
    for patch in bp2['boxes']:
        patch.set_facecolor('lightcoral')
    ax2.set_xlabel('Year', fontsize=11)
    ax2.set_ylabel('Excess Word Ratio (%)', fontsize=11)
    ax2.set_title(f'[{field_name}] Distribution of Excess Word Ratio', fontsize=12)
    ax2.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"저장: {output_path}")


def plot_yearly_mean_trend(analysis_df, output_path, field_name=""):
    """연도별 평균 과잉 단어 사용량 추이"""
    yearly_stats = analysis_df.groupby('year').agg({
        'excess_count': ['mean', 'std', 'median'],
        'excess_unique': ['mean', 'std', 'median'],
        'excess_ratio': ['mean', 'std', 'median']
    }).reset_index()

    yearly_stats.columns = ['year',
                            'count_mean', 'count_std', 'count_median',
                            'unique_mean', 'unique_std', 'unique_median',
                            'ratio_mean', 'ratio_std', 'ratio_median']

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    years = yearly_stats['year'].values

 # (1) 평균 고유 과잉 단어 개수
    ax1 = axes[0]
    ax1.plot(years, yearly_stats['unique_mean'], 'b-o', linewidth=2,
             markersize=6, label='Mean')
    ax1.plot(years, yearly_stats['unique_median'], 'b--s', linewidth=1.5,
             markersize=4, alpha=0.7, label='Median')

    ax1.set_xlabel('Year', fontsize=11)
    ax1.set_ylabel('Unique Excess Words', fontsize=11)
    ax1.set_title(f'[{field_name}] Mean Excess Word Count per Abstract', fontsize=12)
    ax1.legend(fontsize=9)
    ax1.grid(True, alpha=0.3)

    n_samples = analysis_df.groupby('year').size().values
    ci_unique = 1.96 * yearly_stats['unique_std'] / np.sqrt(n_samples)
    ax1.fill_between(years,
                 yearly_stats['unique_mean'] - ci_unique,
                 yearly_stats['unique_mean'] + ci_unique,
                 alpha=0.2)
    ax1.set_xticks(years[::2])
    # (2) 평균 과잉 단어 비율
    ax2 = axes[1]
    ax2.plot(years, yearly_stats['ratio_mean'] * 100, 'r-o', linewidth=2,
             markersize=6, label='Mean')
    ax2.plot(years, yearly_stats['ratio_median'] * 100, 'r--s', linewidth=1.5,
             markersize=4, alpha=0.7, label='Median')

    ax2.set_xlabel('Year', fontsize=11)
    ax2.set_ylabel('Excess Word Ratio (%)', fontsize=11)
    ax2.set_title(f'[{field_name}] Mean Excess Word Ratio per Abstract', fontsize=12)
    ax2.legend(fontsize=9)
    ax2.grid(True, alpha=0.3)

    ci_ratio = 1.96 * yearly_stats['ratio_std'] / np.sqrt(n_samples)
    ax2.fill_between(years,
                 (yearly_stats['ratio_mean'] - ci_ratio) * 100,
                 (yearly_stats['ratio_mean'] + ci_ratio) * 100,
                 alpha=0.2, color='red')
    ax2.set_xticks(years[::2])

    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"저장: {output_path}")

    return yearly_stats


def main():
    print("=" * 60)
    print("과잉 단어 사용 추이 분석")
    print("=" * 60)

    # 출력 폴더 생성
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    # 1. 과잉 단어 로드
    print("\n[1] 과잉 단어 로드 중...")
    excess_words, excess_df = load_excess_words(EXCESS_WORDS_FILE)

    # 2. 초록 데이터 로드
    print("\n[2] 초록 데이터 로드 중...")
    abstracts_df = load_abstracts(ABSTRACTS_FILE)

    # 3. 분석 실행
    print("\n[3] 초록별 과잉 단어 분석...")
    analysis_df = analyze_all_abstracts(abstracts_df, excess_words)

    # 분석 결과 저장
    analysis_df.to_csv(OUTPUT_DIR / 'abstract_excess_analysis.csv',
                       index=False, encoding='utf-8-sig')
    print(f"  → 저장: {OUTPUT_DIR / 'abstract_excess_analysis.csv'}")

    # 4. 방법 2: N개 이상 사용 비율
    print("\n[4] 임계값별 비율 계산...")
    stats_df = calculate_threshold_stats(analysis_df, THRESHOLDS)
    stats_df.to_csv(OUTPUT_DIR / 'threshold_stats.csv',
                    index=False, encoding='utf-8-sig')
    print(stats_df.to_string())

    # 5. 시각화
    print("\n[5] 시각화...")
    plot_threshold_trends(stats_df, THRESHOLDS,
                          OUTPUT_DIR / 'threshold_trends.png', FIELD_NAME)
    plot_score_distribution(analysis_df,
                            OUTPUT_DIR / 'score_distribution.png', FIELD_NAME)
    yearly_stats = plot_yearly_mean_trend(analysis_df,
                                          OUTPUT_DIR / 'yearly_mean_trend.png', FIELD_NAME)

    # 연도별 통계 저장
    yearly_stats.to_csv(OUTPUT_DIR / 'yearly_stats.csv',
                        index=False, encoding='utf-8-sig')

    print("\n" + "=" * 60)
    print("✅ 분석 완료!")
    print(f"   결과 저장 위치: {OUTPUT_DIR}")
    print("=" * 60)

    # 요약 통계 출력
    print("\n[요약]")
    print(f"- 전체 초록 수: {len(analysis_df):,}")
    print(f"- 과잉 단어 1개 이상 사용한 초록: {(analysis_df['excess_unique'] >= 1).sum():,} "
          f"({(analysis_df['excess_unique'] >= 1).mean()*100:.1f}%)")
    print(f"- 과잉 단어 5개 이상 사용한 초록: {(analysis_df['excess_unique'] >= 5).sum():,} "
          f"({(analysis_df['excess_unique'] >= 5).mean()*100:.1f}%)")
    print(f"- 평균 과잉 단어 개수: {analysis_df['excess_unique'].mean():.2f}")

    # 2024년 vs 2020년 비교
    if 2024 in analysis_df['year'].values and 2020 in analysis_df['year'].values:
        y2020 = analysis_df[analysis_df['year'] == 2020]['excess_unique'].mean()
        y2024 = analysis_df[analysis_df['year'] == 2024]['excess_unique'].mean()
        print(f"\n- 2020년 평균: {y2020:.2f} → 2024년 평균: {y2024:.2f} "
              f"(변화: {(y2024-y2020)/y2020*100:+.1f}%)")


if __name__ == '__main__':
    main()

과잉 단어 사용 추이 분석

[1] 과잉 단어 로드 중...
과잉 단어 196개 로드 완료

[2] 초록 데이터 로드 중...
초록 데이터 511787건 로드 완료

[3] 초록별 과잉 단어 분석...

초록 분석 중...
  진행: 0 / 511,787 (0.0%)
  진행: 10,000 / 511,787 (2.0%)
  진행: 20,000 / 511,787 (3.9%)
  진행: 30,000 / 511,787 (5.9%)
  진행: 40,000 / 511,787 (7.8%)
  진행: 50,000 / 511,787 (9.8%)
  진행: 60,000 / 511,787 (11.7%)
  진행: 70,000 / 511,787 (13.7%)
  진행: 80,000 / 511,787 (15.6%)
  진행: 90,000 / 511,787 (17.6%)
  진행: 100,000 / 511,787 (19.5%)
  진행: 110,000 / 511,787 (21.5%)
  진행: 120,000 / 511,787 (23.4%)
  진행: 130,000 / 511,787 (25.4%)
  진행: 140,000 / 511,787 (27.4%)
  진행: 150,000 / 511,787 (29.3%)
  진행: 160,000 / 511,787 (31.3%)
  진행: 170,000 / 511,787 (33.2%)
  진행: 180,000 / 511,787 (35.2%)
  진행: 190,000 / 511,787 (37.1%)
  진행: 200,000 / 511,787 (39.1%)
  진행: 210,000 / 511,787 (41.0%)
  진행: 220,000 / 511,787 (43.0%)
  진행: 230,000 / 511,787 (44.9%)
  진행: 240,000 / 511,787 (46.9%)
  진행: 250,000 / 511,787 (48.8%)
  진행: 260,000 / 511,787 (50.8%)
  진행: 270,000 / 511,787 (52.8%)
